<a href="https://colab.research.google.com/github/brokentwigs/Colab-Analysis/blob/main/SklearnSoftmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
import pandas as pd
from datetime import datetime
from collections import Counter
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from google.colab import drive
enco = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
enco_labels_train = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

In [ ]:
df = pd.read_csv('BPI_Challenge_2012.csv')

FileNotFoundError: ignored

In [ ]:
# Temporal Split
df = df.sort_values(by = ['startTime'])
temporal_test_length = len(df['startTime'].unique()) // 10
temporal_train_length = len(df['startTime'].unique()) - (len(df['startTime'].unique()) // 10)
training_dates = df['startTime'].unique()[0 : temporal_train_length]
testing_dates = df['startTime'].unique()[temporal_train_length:]
df_date_train = df[df['startTime'].isin(training_dates)]
df_date_train
df_date_test = df[df['startTime'].isin(testing_dates)]
df_date_test

# Case Split
testing_length = len(df['case'].unique()) // 10
training_length = len(df['case'].unique()) - (len(df['case'].unique()) // 10)
training_cases = df['case'].unique()[0:training_length]
testing_cases = df['case'].unique()[training_length:]
df_bpi_train = df[df['case'].isin(training_cases)]
df_bpi_train
df_bpi_test = df[df['case'].isin(testing_cases)]
df_date_train.tail(1), df_date_test.head(1)

training_set =  df_bpi_train[df_bpi_train['startTime'].isin(training_dates)]
testing_set = df_bpi_test[df_bpi_test['startTime'].isin(testing_dates)]

train_val_split = 80

final_train = training_set.head(int(len(df)*(train_val_split/100)))

final_val = training_set.tail(len(training_set) - len(final_train))

In [ ]:
final_test = testing_set.sort_values(by = ['case', 'startTime'])
final_val = final_val.sort_values(by = ['case', 'startTime'])
final_train = final_train.sort_values(by = ['case', 'startTime'])

In [ ]:
def preprocess(df):
  df['startTime'] = pd.to_datetime(df['startTime'])
  df['completeTime'] = pd.to_datetime(df['completeTime'])
  df['next_event'] = df['event'].shift(-1)
  d_last = df.index.to_series().groupby(df['case']).agg(['last']).reset_index() # Get indexes of last event in each case
  last_lst = d_last['last'].to_list() # Indexes of last event to list
  for num in last_lst:
    df.loc[num, 'next_event'] = 'no event' # Replace last events in each case to NaN

  df['time_diff'] = df['startTime'].diff().dt.total_seconds().shift(-1)
  d_last = df.index.to_series().groupby(df['case']).agg(['last']).reset_index() # Get indexes of last event in each case
  last_lst = d_last['last'].to_list() # Indexes of last event to list
  for num in last_lst:
    df.loc[num, 'time_diff'] = 0 # Replace last events in each case to 0

  df['position'] = df.groupby(['case']).cumcount()

  return df

In [ ]:
preprocess(final_train)
preprocess(final_test)
preprocess(final_val)

In [ ]:
# one hot encode current event
def OneHot(df):
  to_be_enc = df['event'].to_numpy().reshape(-1, 1)
  encode = enco.fit(to_be_enc)
  transformed = encode.transform(to_be_enc)
  ohe_df = pd.DataFrame(transformed, columns = encode.get_feature_names())

  return ohe_df

In [ ]:
val_ohe = OneHot(final_val)
test_ohe = OneHot(final_test)
train_ohe = OneHot(final_train)

In [ ]:
# normalize time

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
final_train['normalized_time'] = mms.fit_transform(final_train['time_diff'].to_numpy().reshape(-1, 1))
final_test['normalized_time'] = mms.transform(final_test['time_diff'].to_numpy().reshape(-1, 1))
final_val['normalized_time'] = mms.transform(final_val['time_diff'].to_numpy().reshape(-1, 1))

In [ ]:
def weekday(df, df_1):
  df_1.index = df.index
  df['weekday'] = df['startTime'].dt.dayofweek
  df_1['weekday'] = df['weekday']
  weekend = (df_1['weekday'] == 5) | (df_1['weekday'] == 6)

  # 0 if weekend, 1 if weekday
  df_1['weekend'] = np.where(weekend, 0, 1)
  df_1 = df_1.drop('weekday', axis = 1)

  return df_1

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_train['normalized_req'] = scaler.fit_transform(final_train['AMOUNT_REQ'].to_numpy().reshape(-1, 1))
final_test['normalized_req'] = scaler.transform(final_test['AMOUNT_REQ'].to_numpy().reshape(-1, 1))
final_val['normalized_req'] = scaler.transform(final_val['AMOUNT_REQ'].to_numpy().reshape(-1, 1))
final_train['nor_position'] = scaler.transform(final_train['position'].to_numpy().reshape(-1, 1))
final_test['nor_position'] = scaler.transform(final_test['position'].to_numpy().reshape(-1, 1))
final_val['nor_position'] = scaler.transform(final_val['position'].to_numpy().reshape(-1, 1))

In [ ]:
val_ohe = weekday(final_val, val_ohe)
val_ohe['normalized_req'] = final_val['normalized_req']
val_ohe['nor_position'] = final_val['nor_position']

In [ ]:
train_ohe = weekday(final_train, train_ohe)
train_ohe['normalized_req'] = final_train['normalized_req']
train_ohe['nor_position'] = final_train['nor_position']
train_ohe

In [ ]:
test_ohe = weekday(final_test, test_ohe)
test_ohe['normalized_req'] = final_test['normalized_req']
test_ohe['nor_position'] = final_test['nor_position']

In [ ]:
labels_train = np.array(final_train['next_event'])
labels_test = np.array(final_test['next_event'])
labels_val = np.array(final_val['next_event'])

In [ ]:
features_train = np.array(train_ohe)
features_test = np.array(test_ohe)
features_val = np.array(val_ohe)

In [ ]:
to_be_enc_labels_train = labels_train.reshape(-1, 1)
encode_labels_train = enco_labels_train.fit(to_be_enc_labels_train)
transformed_labels_train = encode_labels_train.transform(to_be_enc_labels_train)
ohe_labels_train = pd.DataFrame(transformed_labels_train, columns=encode_labels_train.get_feature_names())

len(ohe_labels_train)

In [ ]:
X = features_train
y = labels_train
X_test = features_test
y_test = labels_test

In [ ]:
#X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)

#print('X_train dimension= ', X_train.shape)
#print('X_test dimension= ', X_test.shape)
#print('y_train dimension= ', y_train.shape)
#print('y_train dimension= ', y_test.shape)

In [ ]:
lm = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')
lm.fit(X, y)

In [ ]:
lm.score(X_test, y_test)

In [ ]:
lm.predict(X_test)

In [ ]:
y_true = y_test
y_pred = lm.predict(X_test)
list_events = final_test['next_event'].unique()
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm,
                     index = list_events,
                     columns = list_events)
import matplotlib.pyplot as plot
import seaborn as sb
plot.figure(figsize=(15,14))
sb.heatmap(cm_df, annot=True)
plot.title('Confusion Matrix')
plot.ylabel('Actal Values')
plot.xlabel('Predicted Values')
plot.show()